In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
from IPython.display import clear_output

import quick_pp.las_handler as las
from quick_pp.lithology import multi_mineral_model
from quick_pp.plotter.well_plots import plotly_log
from quick_pp.qaqc.qaqc import badhole_flag, mask_outside_threshold

from quick_pp.config import Config

In [3]:
with open('X10_raw.las', 'rb') as f:
    df, header, _ = las.read_las_file(f)

# Mask outside threshold
df = mask_outside_threshold(df, True)

# Flag bad hole
df = badhole_flag(df) if 'CALI' in df.columns else df
clear_output()

In [4]:
neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB']].dropna()
vol_quartz, vol_calcite, vol_dolomite, vol_shale, vol_mud = multi_mineral_model(
    neu_den_df['GR'], neu_den_df['NPHI'], neu_den_df['RHOB'])
df_ssc_model = pd.DataFrame(
    {'VSAND': vol_quartz, 'VCLW': vol_shale},
    index=neu_den_df.DEPTH
)
df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

c:\projects\quick_pp\venv310\lib\site-packages\scipy\optimize\_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [5]:
fig = plotly_log(df_ssc)
fig.show()
# fig.write_html('plot.html')